# Projeto 2 - Ciência dos Dados

**Nome**: Jonas da Silva Lopes

**Nome**: William Silva


# Classificador automático de sentimento

## Preparando o ambiente no jupyter:

In [23]:
%%capture
!pip install tweepy

In [24]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re 
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@William48253649***

In [25]:
#Identificador da conta no twitter: @William48253649

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [26]:
#Produto escolhido
produto = 'Nubank'

#Quantidade mínima de mensagens capturadas
n = 500
#Quantidade mínima de mensagens para a base de treinamento
t = 300

#Filtro de língua
lang = 'pt'

Capturando os dados do twitter:

In [27]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower()) #Lower para deixar tudo minúsculo e facilitar a comparação
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [87]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])}).set()
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])}).set()
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Após realizar a classificação manual das mensagens, como irrelevante (0) ou relevante (1), partimos para  a mudança desses valores para algo mais palpável para a análise. Com isso, fazemos a alteração que pode ser vista abaixo.

Mas, para realizar isso, primeiro foi necessário determinar critérios para a classificação:

- Mencionar o produto;
- A menção ao produto deve ser acompanhada de uma opinião;
- A opinião pode ser demonstrada na forma de indagações, reclamações, pode envolver sarcasmo, elogios e sugestões sobre serviços;
- A opinião afirmada deve ser clara;
- Emoctions também representam opiniões.

In [88]:
mensagens = pd.read_excel("Nubank.xlsx")
mensagens.Relevância = mensagens.Relevância.astype('category')
mensagens.Relevância.cat.categories = ('Irrelevante', 'Relevante')

print("A quantidade de cada cada um é: \n\n", mensagens.Relevância.value_counts())

relevante = mensagens[mensagens.Relevância=="Relevante"]
irrelevante = mensagens[mensagens.Relevância=="Irrelevante"]

A quantidade de cada cada um é: 

 Irrelevante    184
Relevante      116
Name: Relevância, dtype: int64


In [90]:
''' Função que troca pontuação por espaço '''
def cleanup(text):
    punctuation = '[!\-.:?;/,|@"\'()]'
    pattern = re.compile(punctuation)
    # Abaixo, determina que se troca por espaço
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

# Usando a função apply para fazer a limpeza nas mensagens
nubank_relev = relevante.Treinamento.apply(cleanup)
nubank_irrelev = irrelevante.Treinamento.apply(cleanup)

In [91]:
# Pegando as palavras em cada tópico: Relevante e Irrelevante

words_relev = pd.DataFrame("".join(nubank_relev).split())
#print(words_relev[0].value_counts())


words_irrelev = pd.DataFrame("".join(nubank_irrelev).split())
print(words_irrelev[0].value_counts())

nubank            147
o                  68
de                 67
e                  60
que                60
https              58
t                  58
co                 58
a                  56
não                52
é                  38
um                 36
da                 33
com                32
do                 31
eu                 30
se                 30
gente              29
tem                28
pra                26
no                 25
meu                25
cartão             24
em                 24
na                 24
você               23
só                 22
conta              22
uma                22
me                 19
                 ... 
zq6cca3lv1          1
d…e                 1
todos               1
vendi               1
corre               1
droga               1
impressionada       1
noçã…eu             1
wj5ukyfgfv          1
zma7gokeer          1
experiência         1
castrocastrado      1
tranferi            1
ifg                 1
salve     

##### Frequência Absoluta

In [11]:
words_irrelev[0].value_counts()

nubank            146
o                  68
de                 67
e                  60
que                60
https              58
t                  58
co                 58
a                  56
não                52
é                  38
um                 36
da                 33
com                32
do                 31
se                 30
eu                 29
gente              29
tem                28
pra                26
no                 25
meu                25
em                 23
cartão             23
na                 23
você               23
uma                22
só                 22
conta              22
limite             19
                 ... 
desistir            1
hahahaha            1
zma7gokeer          1
virtual             1
experiência         1
primeiro            1
castrocastrado      1
tranferi            1
ifg                 1
salve               1
br                  1
conferido           1
rwdhdtvvhe          1
xz90obf2um          1
delelista 

##### Frequência relativa

In [10]:
words_relev[0].value_counts(True)

nubank            0.053656
o                 0.027066
de                0.025641
e                 0.024691
que               0.021368
eu                0.018044
a                 0.016144
me                0.013770
https             0.013295
co                0.013295
meu               0.013295
t                 0.013295
um                0.012821
não               0.012346
é                 0.011871
cartão            0.010921
do                0.010921
no                0.009497
uma               0.007597
pra               0.007123
q                 0.007123
na                0.007123
com               0.006648
só                0.006173
limite            0.005698
to                0.005698
mais              0.005698
nunca             0.005223
tem               0.005223
da                0.005223
                    ...   
né                0.000475
odeio             0.000475
estornou          0.000475
lembrando         0.000475
eveeeeer          0.000475
ela               0.000475
c

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**